In [1]:
from pathlib import Path
from helpers import read_data_to_dict
from helpers import list_entries_recusive

previous_ver = "1.0.3"
new_ver = "1.0.3.1"

previous_ver_en = Path("data") / previous_ver / "Localization" / "en"
new_en_dir = Path("data") / new_ver / "Localization" / "en"


def read_localizatoin_dir_to_dict(localization_dir_path: Path) -> tuple[dict, list]:
    """Read localization directory to dictionary

    Args:
        localization_dir_path (Path): Path to localization directory

    Returns:
        tuple[dict, list]: Tuple of dictionary and list of exceptions

        Dictionary like {"path/to/file": {"game text key": "game text"}, ...}
    """
    entries = list_entries_recusive(localization_dir_path)
    result = {}
    exceptions = []

    for entry in entries:
        if entry.is_dir() or not entry.name.endswith(".loc"):
            print(f"Skipping {entry}")
            continue

        key = entry.relative_to(localization_dir_path).as_posix()
        value = entry.read_text()

        try:
            value_dict = read_data_to_dict(value)
            result[key] = value_dict
        except Exception:
            exceptions.append(key)

    return result, exceptions


previous_en_dict, previous_en_exceptions = read_localizatoin_dir_to_dict(previous_ver_en)
new_en_dict, new_en_exceptions = read_localizatoin_dir_to_dict(new_en_dir)


Skipping data/1.0.3/Localization/en/Journal
Skipping data/1.0.3/Localization/en/Dialogues
Skipping data/1.0.3/Localization/en/Letters
Skipping data/1.0.3/Localization/en/Dialog
Skipping data/1.0.3/Localization/en/Dialog/Footprint
Skipping data/1.0.3/Localization/en/Dialog/Perceptible
Skipping data/1.0.3/Localization/en/Dialog/Objects
Skipping data/1.0.3/Localization/en/Dialog/Corpses
Skipping data/1.0.3/Localization/en/Dialogue
Skipping data/1.0.3/Localization/en/NPC_Names
Skipping data/1.0.3/Localization/en/GUI
Skipping data/1.0.3/Localization/en/HUD
Skipping data/1.0.3.1/Localization/en/Journal
Skipping data/1.0.3.1/Localization/en/Dialogues
Skipping data/1.0.3.1/Localization/en/Letters
Skipping data/1.0.3.1/Localization/en/Dialog
Skipping data/1.0.3.1/Localization/en/Dialog/Footprint
Skipping data/1.0.3.1/Localization/en/Dialog/Perceptible
Skipping data/1.0.3.1/Localization/en/Dialog/Objects
Skipping data/1.0.3.1/Localization/en/Dialog/Corpses
Skipping data/1.0.3.1/Localization/en/D

In [2]:
previous_en_exceptions, new_en_exceptions

(['GUI/UserReport_en.loc'], ['GUI/UserReport_en.loc'])

In [3]:
def diff_dict_key_values(old: dict, new: dict) -> dict:
    difference = {
        "key_added": [],
        "key_removed": [],
        "value_updated": [],
    }
    for key, value in new.items():
        if key not in old:
            difference["key_added"].append(key)
        else:
            predict_value = old[key]
            if value != predict_value:
                difference["value_updated"].append(key)

    for key in old.keys():
        if key not in new:
            difference["key_removed"].append(key)

    return difference


# test
predict = {
    "a": "fooooo",
    "b": "removed!",
    "d": "removed!!",
}
actual = {
    "a": "foo, change!",
    "c": "new!",
}
res = diff_dict_key_values(predict, actual)
print(res)




def has_diff(old: dict, new: dict) -> bool:
    return diff_dict_key_values(old, new) != {"key_added": [], "key_removed": [], "value_updated": []}


# test
print(has_diff(predict, actual))
print(has_diff(predict, predict))

{'key_added': ['c'], 'key_removed': ['b', 'd'], 'value_updated': ['a']}
True
False


In [18]:
# Search Differences
new_files = []
differences = {}


for new_file_key, new_file_content in new_en_dict.items():
    if new_file_key not in previous_en_dict.keys():
        print(f"New file found: {new_file_key}")
        new_files.append(new_file_key)
    else:
        previous_file_content = previous_en_dict[new_file_key]

        diff = diff_dict_key_values(previous_file_content, new_file_content)
        if diff == {"key_added": [], "key_removed": [], "value_updated": []}:
            # print(f"No diff found in {new_file}")
            continue
        else:
            print(f"Diff found in {new_file_key}")
            differences[new_file_key] = diff


print()
print("### New files\n", new_files)
print()
print("### Differences\n", differences)


Diff found in Journal/TraderLostItem_en.loc
Diff found in Buffs_en.loc

### New files
 []

### Differences
 {'Journal/TraderLostItem_en.loc': {'key_added': ['Entry_03_Cancel'], 'key_removed': [], 'value_updated': []}, 'Buffs_en.loc': {'key_added': [], 'key_removed': [], 'value_updated': ['BleedingDescription']}}


In [19]:
print(differences)

{'Journal/TraderLostItem_en.loc': {'key_added': ['Entry_03_Cancel'], 'key_removed': [], 'value_updated': []}, 'Buffs_en.loc': {'key_added': [], 'key_removed': [], 'value_updated': ['BleedingDescription']}}
